# Data Preparation and Analysis Draft
## Flow change by hour, by day, by month, by year, by route



In [3]:
import os
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import streamlit as st
from scipy import stats
from deta import Deta

In [15]:
# download data from Deta
deta = Deta(os.environ['deta_key'])
test_drive = deta.Drive("bp_data")
d12_hr_MLHVflow_201902_csv = test_drive.get('d12_hr_MLHVflow_201902.csv')
with open('data/d12_hr_MLHVflow_201902.csv', 'wb+') as f:
    for chunk in d12_hr_MLHVflow_201902_csv.iter_chunks(4096):
        f.write(chunk)
    d12_hr_MLHVflow_201902_csv.close()

In [17]:
df_flow = (pd
 .read_csv('data/d12_hr_MLHVflow_201902.csv', parse_dates=['timestamp'])
 .assign(timestamp=lambda df_: df_['timestamp'].dt.tz_localize(None))
 .assign(year=lambda df_: df_['timestamp'].dt.year,
         month=lambda df_: df_['timestamp'].dt.month,
         day=lambda df_: df_['timestamp'].dt.day,
         hour=lambda df_: df_['timestamp'].dt.hour)
)

In [32]:
route_numbers = df_flow['route'].unique()

In [20]:
type(route_numbers)

numpy.ndarray

In [33]:
route_numbers = np.sort(route_numbers)
route_numbers

array([  1,   5,  22,  55,  57,  73,  74,  91, 133, 142, 241, 261, 405,
       605], dtype=int64)

In [22]:
# a function to clean the raw flow df
# use route number as input
# convert to pivot table with index of station_id and abs_pm, column of hour
# drop the outliers (z-score > 3)
# values are mean of flow throughout 28 days in 2019.02
def route_df(route_num):
    route_raw = df_flow[(df_flow.route==route_num)]
    pivot_raw = pd.pivot_table(route_raw, values='total_flow', index=['station_id', 'abs_pm'], columns=['hour'], aggfunc=np.mean).sort_values('abs_pm')
    route_raw_new = pivot_raw[(np.abs(stats.zscore(pivot_raw)) < 3)].dropna().reset_index()
    route_flow_df = route_raw[route_raw['station_id'].isin(route_raw_new['station_id'])]
    route_flow_df = route_flow_df.assign(day_of_week=lambda x: (x['timestamp'].dt.dayofweek))
    route_flow_df['weekday'] = route_flow_df['day_of_week'].apply(lambda x: 'weekday' if x < 6 else 'weekends')
    return route_flow_df
route_74 = route_df(74)

In [35]:
route_df_list = []
for i in route_numbers:
    route_df_list.append(route_df(i))

## Plot flow for each route by day and time

- x=hour; y=day


In [37]:
# plot both direction flow by hour
def weekday_pivot_df(route_df, route_num):
    direction_a = route_df['direction'].unique()[0]
    direction_b = route_df['direction'].unique()[1]
    route_df_direction_a_pivot = pd.pivot_table(route_df[route_df['direction']==direction_a], values='total_flow', index=['hour'], columns=['weekday'], aggfunc=np.mean).reset_index()
    route_df_direction_b_pivot = pd.pivot_table(route_df[route_df['direction']==direction_b], values='total_flow', index=['hour'], columns=['weekday'], aggfunc=np.mean).reset_index()
    route_df_pivot = route_df_direction_a_pivot.add(route_df_direction_b_pivot, fill_value=0)
    route_df_pivot['hour'] = route_df_pivot['hour']/2
    route_df_pivot = route_df_pivot.round(0)
    route_df_pivot['route'] = route_num
    return route_df_pivot

In [41]:
weekday_flow_pivot_list = []
for i in range(0, len(route_numbers)):
    weekday_flow_pivot_list.append(weekday_pivot_df(route_df_list[i], route_numbers[i]))

In [45]:
all_pivot_df = pd.concat(weekday_flow_pivot_list)
all_pivot_df

weekday,hour,weekday,weekends,route
0,0.0,840.0,1350.0,1
1,1.0,540.0,849.0,1
2,2.0,423.0,619.0,1
3,3.0,436.0,407.0,1
4,4.0,953.0,395.0,1
...,...,...,...,...
19,19.0,5338.0,4540.0,605
20,20.0,4275.0,3976.0,605
21,21.0,3711.0,3373.0,605
22,22.0,2788.0,2492.0,605


In [48]:
all_pivot_df.to_csv('all_pivot_df.csv', index=False)

In [56]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=route_55_pivot["hour"],
        y=route_55_pivot["weekday"],
        name="55",
        line=dict(color="royalblue", width=4),
    )
)
fig.add_trace(
    go.Scatter(
        x=route_57_pivot["hour"],
        y=route_57_pivot["weekday"],
        name="57",
        line=dict(color="lime", width=4),
    )
)

In [57]:
fig = go.Figure()
for i in range(0, len(weekday_flow_pivot_list)):
    fig.add_trace(
        go.Scatter(
            x=weekday_flow_pivot_list[i]['hour'],
            y=weekday_flow_pivot_list[i]['weekday'],
            name=str(weekday_flow_pivot_list[i]['route'][0])))
fig.show()

In [58]:
fig = go.Figure()
for i in range(0, len(weekday_flow_pivot_list)):
    fig.add_trace(
        go.Scatter(
            x=weekday_flow_pivot_list[i]['hour'],
            y=weekday_flow_pivot_list[i]['weekends'],
            name=str(weekday_flow_pivot_list[i]['route'][0])))
fig.show()